In [19]:
import duckdb
from pathlib import Path

DB_PATH = Path(r"C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\analytics.duckdb")
con = duckdb.connect(str(DB_PATH), read_only=True)

print("rows gold:", con.execute("SELECT COUNT(*) FROM gold.dataset_modelo").fetchone()[0])
print("rows clientes:", con.execute("SELECT COUNT(*) FROM silver_v2.clientes").fetchone()[0])

con.close()

rows gold: 63257
rows clientes: 56993


In [21]:
import duckdb
from pathlib import Path
import datetime as dt

DB_PATH = Path(r"C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\analytics.duckdb")
OUT_DIR = Path(r"C:\Users\serrios\01_prueba_analitico_4\op_cobro\notebooks\outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Si quieres evitar bloqueos por archivo abierto, usa nombre con timestamp:
ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
out_file = OUT_DIR / f"gold_dataset_modelo_{ts}.csv"

# Si prefieres SIEMPRE el mismo nombre, usa esto:
# out_file = OUT_DIR / "gold_dataset_modelo.csv"

# Si el archivo ya existía, intentamos borrarlo (si está abierto, aquí fallará con PermissionError)
if out_file.exists():
    out_file.unlink()

try:
    with duckdb.connect(str(DB_PATH), read_only=True) as con:
        con.execute(f"""
            COPY (SELECT * FROM gold.dataset_modelo)
            TO '{out_file.as_posix()}'
            (HEADER, DELIMITER ';');
        """)
    print("✅ Exportado:", out_file)

except Exception as e:
    print("❌ Falló el export. Detalle del error:")
    print(repr(e))
    print("\n🔎 Checklist rápido:")
    print("1) ¿El CSV está abierto en Excel/PowerBI? Ciérralo.")
    print("2) ¿Existe la carpeta outputs y tienes permisos?")
    print("3) Prueba cambiar DELIMITER ',' si Excel no lo requiere.")
    print("4) Prueba exportar a Parquet (más estable) si el dataset es grande.")

✅ Exportado: C:\Users\serrios\01_prueba_analitico_4\op_cobro\notebooks\outputs\gold_dataset_modelo_20260223_002035.csv
